# Setup

In [12]:
# Environment setup
import pandas as pd
# import requests
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env from the project root (parent of data_cleaning)
load_dotenv(Path.cwd().resolve().parent / ".env")
GOOGLE_SHEETS_API_KEY = os.getenv('GOOGLE_SHEETS_API_KEY')
PROJECTS_SHEET_ID = os.getenv('PROJECTS_SHEET_ID')
ACTIVES_SHEET_ID = os.getenv('ACTIVES_SHEET_ID')
FALL_ATTENDANCE_SHEET_ID = os.getenv('FALL_ATTENDANCE_SHEET_ID')
BASE_URL = "https://sheets.googleapis.com/v4/spreadsheets"
load_dotenv(Path.cwd().resolve().parent / ".env")

True

In [64]:
#def get_sheet_titles(spreadsheet_id: str, api_key: str) -> list[str]:
#    """Return all sheet/tab titles in the spreadsheet."""
#    resp = requests.get(
#        f"{BASE_URL}/{spreadsheet_id}",
#        params={
#            "fields": "sheets(properties(title))",
#            "key": api_key,
#        },
#        timeout=30,
#    )
#    resp.raise_for_status()
#    data = resp.json()
#    return [s["properties"]["title"] for s in data.get("sheets", [])]

In [65]:
#def fetch_values_batch(spreadsheet_id: str, api_key: str, sheet_titles: list[str]) -> dict[str, list[list]]:
#    """Batch fetch values for provided sheet titles.
#    Returns mapping of title -> 2D list of cell values (including header row).
#    """
#    if not sheet_titles:
#        return {}
#    # Multiple 'ranges' params are supported by the API
#    params = [("key", api_key), ("valueRenderOption", "UNFORMATTED_VALUE"), ("dateTimeRenderOption", "FORMATTED_STRING")]
#    params.extend(("ranges", title) for title in sheet_titles)
#    resp = requests.get(
#        f"{BASE_URL}/{spreadsheet_id}/values:batchGet",
#        params=params,
#        timeout=60,
#    )
#    resp.raise_for_status()
#    payload = resp.json()

#    values_by_title: dict[str, list[list]] = {}
#    for vr, title in zip(payload.get("valueRanges", []), sheet_titles):
#        values_by_title[title] = vr.get("values", [])
#    return values_by_title

In [66]:
#def values_to_dataframe(values: list[list]) -> pd.DataFrame:
#    """Convert a 2D list from Sheets API to a DataFrame, using first row as header.
#    Pads short rows so all rows match header length.
#    """
#    if not values:
#        return pd.DataFrame()
#    header = [str(h) for h in values[0]]
#    rows = values[1:] if len(values) > 1 else []
#    normalized_rows = [row + [""] * (len(header) - len(row)) for row in rows]
#    return pd.DataFrame(normalized_rows, columns=header)

In [67]:
#def fetch_spreadsheet_as_dataframes(spreadsheet_id: str, api_key: str) -> dict[str, pd.DataFrame]:
#    """Fetch all sheets in a spreadsheet and return {sheet_title: DataFrame}."""
#    titles = get_sheet_titles(spreadsheet_id, api_key)
#    if not titles:
#        return {}
#    values_by_title = fetch_values_batch(spreadsheet_id, api_key, titles)
#    return {title: values_to_dataframe(values_by_title.get(title, [])) for title in titles}

# Load data

In [13]:
# # fetch all data from Google Sheets into DataFrames per sheet
# # Build DataFrames per sheet for each spreadsheet ID found in .env
# PROJECTS_DFS = fetch_spreadsheet_as_dataframes(PROJECTS_SHEET_ID, GOOGLE_SHEETS_API_KEY) if PROJECTS_SHEET_ID else {}
# ACTIVES_DFS = fetch_spreadsheet_as_dataframes(ACTIVES_SHEET_ID, GOOGLE_SHEETS_API_KEY) if ACTIVES_SHEET_ID else {}
# FALL_ATTENDANCE_DFS = fetch_spreadsheet_as_dataframes(FALL_ATTENDANCE_SHEET_ID, GOOGLE_SHEETS_API_KEY) if FALL_ATTENDANCE_SHEET_ID else {}
# Load data from CSV URLs exported from Google Sheets
projects_df = pd.read_csv(os.getenv('PROJECTS_SHEET_URL'))
actives_df = pd.read_csv(os.getenv('ACTIVES_SHEET_URL'))
attendance_df = pd.read_csv(os.getenv('ATTENDANCE_URL'))

In [14]:
actives_df.head()

,Timestamp,Name,Year,Are you planning to be an active member this quarter? (All actives have to pay dues)
0,10/2/2025 19:37:26,Spencer,3rd,Yes
1,10/2/2025 19:37:27,Jayden Patel,2nd,Yes
2,10/2/2025 19:39:01,Mattie freaking dao,2nd,Yes
3,10/2/2025 19:42:19,aastha,3rd,Yes
4,10/2/2025 19:42:52,Mihir Joshi,4th,Yes


In [15]:
actives_df['Timestamp'] = pd.to_datetime(actives_df['Timestamp'])
actives_df['Name'] = actives_df['Name'].str.lower()
actives_df['Year'] = actives_df['Year'].map({'1st': 1, '2nd': 2, '3rd': 3, '4th': 4, '5th': 5}).fillna(actives_df['Year'])
actives_df['Are you planning to be an active member this quarter? (All actives have to pay dues)'] = actives_df.iloc[:, 3].apply(lambda x: True if x == 'Yes' else False)

In [16]:
actives_df.dtypes

Timestamp                                                                               datetime64[ns]
Name                                                                                            object
Year                                                                                             int64
Are you planning to be an active member this quarter? (All actives have to pay dues)              bool
dtype: object

In [17]:
projects_df['Status'] = projects_df['Status'].fillna("")
projects_df.columns = ['quarter', 'company', 'point_of_contact', 'project_manager', 'associates',
       'nda', 'status', 'description']

In [18]:
projects_df['nda'] = projects_df['nda'].apply(lambda x: True if x == "Yes" else False)

In [19]:
projects_df.head()

,quarter,company,point_of_contact,project_manager,associates,nda,status,description
0,S23,CARI Health,NaN,Ashley Lee & Edmond Hong,NaN,False,,The project focused on two primary objectives:...
1,FA23,UCSD Craft Center,NaN,Sydney Jang,NaN,True,,TCG identified strategies to enhance client en...
2,FA23,MD Revolution,NaN,Dhathry Doppalapudi,NaN,False,,The project focused on completing a competitiv...
3,FA23,Atmo Biosciences,NaN,Isabel Wang,NaN,True,,This project focused on conducting market rese...
4,WI24,Empirical,NaN,Daniel Woo,NaN,True,,Created a list of companies aligning with Empi...


In [20]:
unique_companies = projects_df['company'].unique()
companies_range = range(len(projects_df['company'].unique()))
company_inds = dict(zip(unique_companies, companies_range))
companies_df = pd.DataFrame({"company_id": companies_range, "name": unique_companies})
                            
projects_df['company'] = projects_df['company'].replace(company_inds)

/var/folders/9b/t2jx3nk52wq82g1czdh9gv440000gn/T/ipykernel_33692/3879467409.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  projects_df['company'] = projects_df['company'].replace(company_inds)


# company

In [21]:
companies_df.head()

,company_id,name
0,0,CARI Health
1,1,UCSD Craft Center
2,2,MD Revolution
3,3,Atmo Biosciences
4,4,Empirical


# project

In [22]:
projects_df.head()

,quarter,company,point_of_contact,project_manager,associates,nda,status,description
0,S23,0,NaN,Ashley Lee & Edmond Hong,NaN,False,,The project focused on two primary objectives:...
1,FA23,1,NaN,Sydney Jang,NaN,True,,TCG identified strategies to enhance client en...
2,FA23,2,NaN,Dhathry Doppalapudi,NaN,False,,The project focused on completing a competitiv...
3,FA23,3,NaN,Isabel Wang,NaN,True,,This project focused on conducting market rese...
4,WI24,4,NaN,Daniel Woo,NaN,True,,Created a list of companies aligning with Empi...


# quarter

In [23]:
unique_quarters = projects_df['quarter'].unique()
quarters_range = range(len(projects_df['quarter']))
quarter_inds = dict(zip(unique_quarters, quarters_range))
quarter_df = pd.DataFrame({"quarter_id": quarters_range})
quarter_df.head()

,quarter_id
0,0
1,1
2,2
3,3
4,4


# assignment

In [44]:
assignments = pd.DataFrame(columns=['member_id', 'project_id'])
assignments.head()

,member_id,project_id


# member

In [45]:
members = pd.DataFrame(columns=['member_id', 'name', 'year', 'status', 'role', 'email'])
members['name'] = actives_df['Name']
members['year'] = actives_df['Year']
members['status'] = actives_df['Are you planning to be an active member this quarter? (All actives have to pay dues)'].astype(int)
members['role'] = '-'
members['email'] = '-'
members['member_id'] = members.index
members.head()

,member_id,name,year,status,role,email
0,0,spencer,3,1,-,-
1,1,jayden patel,2,1,-,-
2,2,mattie freaking dao,2,1,-,-
3,3,aastha,3,1,-,-
4,4,mihir joshi,4,1,-,-


# attendance

In [51]:
attendance = pd.DataFrame(columns=['member_id', 'gbm_id', 'status'])
attendance.head()

,member_id,gbm_id,status


# gbm

In [46]:
gbm = pd.DataFrame(columns=['gbm_id', 'quarter_id', 'date'])
gbm.head()

,gbm_id,quarter_id,date


# enrollment

In [47]:
enrollment = pd.DataFrame(columns=['member_id', 'quarter_id'])
enrollment.head()

,member_id,quarter_id
